### Your name:

<pre> Lilian Li </pre>

### Collaborators:

<pre>http://nbviewer.jupyter.org/github/ageron/handson-ml/blob/master/index.ipynb</pre>


In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

Open the housing data


In [2]:
import os
import tarfile
from six.moves import urllib

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

print(DOWNLOAD_ROOT)
print(HOUSING_PATH)
print(HOUSING_URL)

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

fetch_housing_data()
import pandas as pd

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

housing = load_housing_data()
housing.head()





https://raw.githubusercontent.com/ageron/handson-ml/master/
datasets\housing
https://raw.githubusercontent.com/ageron/handson-ml/master/datasets/housing/housing.tgz


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


### Build full pipeline for the data analysis following the example of the notebook.
https://github.com/ageron/handson-ml
- has a great example of density map in California
Hint: the main part requested to change is the algorithm used (Lasso regression)

If you want to learn more about the Lasso regression, see resources below:
- http://scikit-learn.org/stable/modules/linear_model.html#lasso
- https://www.analyticsvidhya.com/blog/2016/01/complete-tutorial-ridge-lasso-regression-python/
- https://stackoverflow.com/questions/40758562/can-anyone-explain-me-standardscaler
Regressions with regularization - as the value of alpha increases, the model complexity reduces. Though higher values of alpha reduce overfitting, significantly high values can cause underfitting as well (eg. alpha = 5). Thus alpha should be chosen wisely. A widely accept technique is cross-validation, i.e. the value of alpha is iterated over a range of values and the one giving higher cross-validation score is chosen.
- lasso performs feature selection

#### Considerations for building pipeline:

- Split data into training and testing sets below.
- Convert all categorical data to one-hot vectors below
- Normalize all non-categorical data 
-  Perform Lasso-based regression using a variety of values for $\alpha$ between 0 and 1 via a grid search where  *housing_labels* is the output and all other features are the input (similar to as seen in lecture two.)

In [7]:
from sklearn.linear_model import Lasso
from sklearn import preprocessing
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.model_selection import train_test_split
from pandas.plotting import scatter_matrix
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import LabelBinarizer

#reinitializing
housing = load_housing_data()
# Write your code here:

#Steps to understand your data
#print(housing.info())
#print(housing["ocean_proximity"].value_counts())
#print(housing.describe())

#splitting test and train set as 0.2 and 0.8, random seed = 18
train_set, test_set = train_test_split(housing, test_size = 0.2, random_state=18)
print(len(train_set), "train +", len(test_set), "test")
#print(train_set["ocean_proximity"].value_counts())

#creating new calculated fields
#housing["id"] = housing["longitude"] * 1000 + housing["latitude"]

imputer = Imputer(strategy="median")
housing_num = housing.drop('ocean_proximity', axis=1)#axis = 1 means columns
imputer.fit(housing_num)

class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names=attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values
    
from sklearn.base import TransformerMixin #gives fit_transform method
class MyLabelBinarizer(TransformerMixin):
    def __init__(self, *args, **kwargs):
        self.encoder = LabelBinarizer(*args, **kwargs)
    def fit(self, x, y=0):
        self.encoder.fit(x)
        return self
    def transform(self, x, y=0):
        return self.encoder.transform(x)
    
    
# column index for dataset with dropped ocean proximity
rooms_ix, bedrooms_ix, population_ix, household_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True): # no *args or **kargs
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
        population_per_household = X[:, population_ix] / X[:, household_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household,
                         bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

#cat and num field names in dataframe
num_attribs = list(housing_num.drop("median_house_value", axis=1))
cat_attribs = ["ocean_proximity"]

        
#defining cat and num pipelines 
num_pipeline = Pipeline([
        ('selector', DataFrameSelector(num_attribs)),
        ('imputer', Imputer(strategy="median")),
        ('attribs_adder', CombinedAttributesAdder()),#add_bedrooms_per_room=False
        ('std_scaler', StandardScaler()),
    ])

#housing_num_tr = num_pipeline.fit_transform(housing_num)
#housing_num_tr = pd.DataFrame(housing_num_tr, columns=list(housing_num.columns)+["rooms_per_household", "population_per_household"],
#                              index = list(housing_num.index.values))
#print(housing_num_tr.head())

cat_pipeline = Pipeline([
        ('selector', DataFrameSelector(cat_attribs)),
        ('label_binarizer',MyLabelBinarizer()),
    ])

housing_cat_test = cat_pipeline.fit_transform(housing)


full_pipeline = FeatureUnion(transformer_list=[
        ("num_pipeline", num_pipeline),
        ("cat_pipeline", cat_pipeline),
    ])

#housing_prepared = full_pipeline.fit_transform(housing)
#print(housing_prepared[1:4])
#print(housing_prepared.shape)

#train set
train_set_nolabel = train_set.drop("median_house_value", axis=1) # drop labels for training set
train_labels = train_set["median_house_value"].copy()

train_set_nolabel_prepared = full_pipeline.fit_transform(train_set_nolabel)

#test set
test_set_nolabel = test_set.drop("median_house_value", axis=1) # drop labels for training set
test_labels = test_set["median_house_value"].copy()

test_set_nolabel_prepared = full_pipeline.fit_transform(test_set_nolabel)


lasso_reg = Lasso(alpha = 1)
lasso_reg.fit(train_set_nolabel_prepared, train_labels)


housing_predictions = lasso_reg.predict(test_set_nolabel_prepared)
housing_labels = test_labels

from sklearn.metrics import mean_squared_error
lasso_mse = mean_squared_error(housing_labels, housing_predictions)
lasso_mse = np.sqrt(lasso_mse)
print(lasso_mse)

#cross validation
#from sklearn.model_selection import cross_val_score
#scores = cross_val_score(lasso_reg, train_set_nolabel_prepared, train_labels,
#                         scoring="mean_squared_error", cv=2)
#lasso_scores = np.sqrt(scores)
#print(lasso_scores)

#def display_scores(scores):
#    print("Scores:", scores)
#    print("Mean:", scores.mean())
#    print("Standard deviation:", scores.std())

#display_scores(lasso_scores)


from sklearn.model_selection import GridSearchCV
model = Lasso()
alphas = np.array([1,0.9,0.7,0.5,0.4,0.2,0.1,0.01,0.001,0.0001,0])
# create and fit a ridge regression model, testing each alpha
grid = GridSearchCV(estimator=model, param_grid=dict(alpha=alphas))
grid.fit(train_set_nolabel_prepared, train_labels)
#print(grid)


# summarize the results of the grid search
print(grid.best_score_)
print(grid.best_estimator_.alpha) #1.0 is the best estimater


#testing 0.1 alpha
lasso_reg = Lasso(alpha = 1.0)
lasso_reg.fit(train_set_nolabel_prepared, train_labels)


housing_predictions = lasso_reg.predict(test_set_nolabel_prepared)
housing_labels = test_labels

from sklearn.metrics import mean_squared_error
lasso_mse = mean_squared_error(housing_labels, housing_predictions)
lasso_mse = np.sqrt(lasso_mse)
print(lasso_mse)

from sklearn.metrics import mean_absolute_error

lin_mae = mean_absolute_error(housing_labels, housing_predictions)
print(lin_mae)

16512 train + 4128 test


C:\Users\theli\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


67940.8095481403


C:\Users\theli\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\theli\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\theli\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\theli\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increa

C:\Users\theli\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\theli\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\theli\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:477: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  positive)
C:\Users\theli\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.


0.6467211469084531
1.0
67940.8095481403
49654.01785852254



Why is it necessary to normalize all continuous variables before performing Lasso? (OPTIONAL)

<pre> it is important because Lasso puts restraints on the coefficients of each variable, the model adds penalty depending on the absolute sum of the coefficients. Normalizing ensures the variables treated on the same scale when optimizing the model and allows the best features to be selected... also helps with interpretability </pre>

### Conclusions
For what values of $\alpha$ does Lasso perform best? Does it perform as well on the housing data as the linear regressor from the lectures? Why do you think this is?

<pre> 1.0 was the best alpha, the performance of the 2 models are about the same, maybe in this scenario, constraining the coefficients/limiting the features selected did not necessarily help the model (since there are not that many variables to begin with) </pre>

### Read appending B

- Reflect on your last data project, read appendix B. Then, write down a few of the checklist items that your last data project could have used. If you have not yet done a data project, then write down a few of the items that you found most interesting.


Last data project could have used:
    - more data prep steps(e.g imputation methods, getting rid of outliers, onehot)
    - crossvalidation to automate and optimize best hyperparams